# 18. 강화 학습

강화 학습(Reinforcement Learning)은 요즘 머신러닝에서 가장 흥미진진한 분야이자 가장 오래된 분야이다. 강화 학습이 현대에 다시 재조명을 받은 시기는 2013년에 딥마인드에서 시도한 <a href="https://arxiv.org/pdf/1312.5602.pdf?source=post_page---------------------------">아타리게임들에 대해 수행된 연구</a>때문입니다. 이 연구에서 기계는 화면 픽셀에 대한 데이터만 입력으로 받고 <a href="https://www.datascienceassn.org/sites/default/files/Human-level%20Control%20Through%20Deep%20Reinforcement%20Learning.pdf">게임 규칙에 대한 어떤 사전 정보없이 대부분 사람을 능가하는 성과</a>를 냈다.

딥마인드가 이러한 성과를 낼수 있던 이유는? 강화 학습 분야에 강력한 딥러닝을 적용했기 때문이다.

18장에서는,
* 강화 학습의 정의 및 활용 분야
* 정책 그라디언트, 심층 Q-네트워크

## 18.1. 보상을 최적화하기 위한 학습

### 강화 학습의 구성요소

* 에이전트: 인공지능 플레이어
* 환경: 에이전트가 솔루션을 찾기 위한 무대
* 행동: 에이전트가 환경 안에서 시행하는 상호작용
* 보상: 에이전트의 행동에 따른 점수 혹은 결과

위의 4가지 요소를 가지고 강화학습은 **'에이전트는 관측을 하고 주어진 환경에서 행동을 하고, 이에 대한 결과로 보상을 받는다'**라는 문장으로 요약할 수 있다. 에이전트는 환경 아래에서 시행착오를 겪으며 보상을 최대로 하는 방향으로 학습한다. 강화학습은 자율주행 자동차, 추천 시스템, 웹페이지에 광고 배치, 이미지 분류 시스템의 제어 등에 사용될 수 있다.

## 18.2. 정책 탐색

에이전트가 행동을 결정하기 위해 사용하는 알고리즘을 정책(policy)라고 한다. 아래의 그림과 같이 Agent가 위치한 상태를 입력으로 받고 행동을 출력하는 신경망이 정책이 될 수 있다.

![RL_figure](../../img/RL_figure.jpg)

### 강화 학습의 예시; 청소기

* Agent: 30분 동안 수집한 먼지의 양을 보상으로 받는 로봇 진공청소기
* 정책: 매 초마다 p의 확률로 전진 or (1-p)의 확률로 왼쪽 또는 오른쪽으로 랜덤하게 회전; 회전의 각도는 -r과 +r 사이의 랜덤한 각도

**어떻게 훈련할 수 있을까?(정책탐색; Policy Search)**
1. 무작위 방식: 정책 파라미터들에 대해 무작위로 시행을 수행하고 성능이 좋은 조합을 선택
2. 유전 알고리즘(Genetic Algorithm): 1세대 정책 100개를 랜덤하여 생성하고, 하위 80개의 정책을 drop. 남은 20개를 활용하여 자식 정책 4개를 생성한다. 자식 정책 4개는 복사된 부모의 정책과 약간의 무작위 성을 설정한 것.
3. 정책 그라디언트(Policy Gradient): 정책 파라미터에 대한 보상의 그라디언트를 평가하여 높은 보상의 방향을 따르는 그라디언트로 파라미터를 수정하는 최적화 기법


## 18.3. OpenAI 짐

강화 학습 에이전트 훈련을 위한 최소한의 시뮬레이션 환경을 제공하는 패키지

간단하게 확인해 볼 환경은 CartPole이라는 아타리의 게임 중 기울어지는 막대를 세우는 게임이다. 

CartPole 환경에서 return되는 관측값은 아래와 같이 구성된다.

[수평 위치(0.0=중앙), 카트의 속도(양수=우측; 음수=좌측), 막대의 각도(0.0=수직), 막대의 각속도(양수=시계방향; 음수=반시계방향)]

In [1]:
import gym
env = gym.make("CartPole-v1")
obs = env.reset()
obs

array([-0.01226301,  0.01158954,  0.0012565 ,  0.03407597])

In [2]:
env.action_space

Discrete(2)

In [3]:
action = 1
obs, reward, done, info = env.step(action)

In [5]:
obs

array([-0.01203122,  0.20669345,  0.00193802, -0.25821025])

In [6]:
reward

1.0

In [7]:
done

False

In [8]:
info

{}

In [13]:
def basic_policy(obs):
    angle = obs[2]
    return 0 if angle < 0 else 1

totals = []
for episode in range(500):
    episode_rewards = 0
    obs = env.reset()
    for step in range(200):
        action = basic_policy(obs)
        obs, reward, done, info = env.step(action)
        episode_rewards += reward
        if done:
            break
    totals.append(episode_rewards)

In [14]:
import numpy as np

totals= np.array(totals)
totals.shape

(500,)

In [17]:
rewards_mean = np.mean(totals)
rewards_std = np.std(totals)
rewards_min = np.min(totals)
rewards_max = np.max(totals)

print("mean:{:.2f}; std:{:.4f}; min:{}; max:{}".format(rewards_mean,rewards_std,rewards_min,rewards_max))

mean:41.52; std:9.0848; min:24.0; max:68.0
